In [29]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
credentials_location = '/Users/xiaozhuxin/.google/credentials/google_credentials.json'

conf = SparkConf() \
    .setMaster('spark://MacBook-Air-2.local:7077') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [ ]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [13]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

25/04/11 21:44:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [14]:
df_green = spark.read.parquet('gs://terraform-introduction-449023-terra-bucket/pq/green/*/*')

In [15]:
df_green.count()

2304517

In [16]:
df_green.registerTempTable('green')

/usr/local/Cellar/apache-spark/3.5.5/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [17]:
df_green_revenue = spark.sql("""
    SELECT 
        -- Revenue grouping 
        date_trunc('hour', lpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM 
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1, 2
    """)

In [18]:
# df_green_revenue.write.mode('overwrite').parquet('data/report/revenue/green/')

In [19]:
df_yellow = spark.read.parquet('gs://terraform-introduction-449023-terra-bucket/pq/yellow/*/*')

In [20]:
df_yellow.registerTempTable('yellow')

In [21]:
df_yellow_revenue = spark.sql("""
    SELECT 
        -- Revenue grouping 
        date_trunc('hour', tpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM 
        yellow
    WHERE
        tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1, 2
    """)

In [22]:
# df_yellow_revenue.write.mode('overwrite').parquet('data/report/revenue/yellow/')

In [23]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [24]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [25]:
common_columns = []

yellow_columns = set(df_yellow.columns)
for col in df_green.columns:
        if col in yellow_columns:
            common_columns.append(col)

In [26]:
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [30]:
df_green_filter = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

In [31]:
df_yellow_filter = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [32]:
df_trips_data = df_green_filter.unionAll(df_yellow_filter)

In [33]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [34]:
df_trips_data.registerTempTable('trips_data')

In [35]:
df_result = spark.sql("""
    SELECT 
        -- Revenue grouping 
        PULocationID AS revenue_zone,
        date_trunc('month', pickup_datetime) AS revenue_month, 
        service_type, 
    
        -- Revenue calculation 
        SUM(fare_amount) AS revenue_monthly_fare,
        SUM(extra) AS revenue_monthly_extra,
        SUM(mta_tax) AS revenue_monthly_mta_tax,
        SUM(tip_amount) AS revenue_monthly_tip_amount,
        SUM(tolls_amount) AS revenue_monthly_tolls_amount,
        SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
        SUM(total_amount) AS revenue_monthly_total_amount,
    
        -- Additional calculations
        AVG(passenger_count) AS avg_monthly_passenger_count,
        AVG(trip_distance) AS avg_monthly_trip_distance
    
    FROM 
        trips_data
    GROUP BY 
        1, 2, 3
    """)

In [36]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|         218|2020-01-01 00:00:00|       green|  24689.350000000126|              1561.75|                  121.5|          

In [36]:
# df_result.coalesce(1).write.mode("overwrite").parquet('data/report/revenue/')

In [37]:
spark.stop()